# Pydantic Tutorial for Beginners

Welcome to the tutorial on **Pydantic** - Python's most popular data validation library!

## What You'll Learn
- Data validation and type safety
- Creating models for complex data structures
- Custom validation logic
- Configuration management
- Building APIs with FastAPI

## Prerequisites
Basic Python knowledge (variables, functions, classes)

Let's get started!

---

# 1. Introduction & Setup

## What is Pydantic?

Pydantic is a data validation library that uses Python type annotations to:
- ✅ Validate data automatically
- ✅ Parse and convert data types
- ✅ Provide clear error messages
- ✅ Generate JSON schemas
- ✅ Power frameworks like FastAPI

## Why Use Pydantic?

Instead of writing manual validation:
```python
def create_user(data):
    if not isinstance(data['name'], str):
        raise ValueError('name must be a string')
    if not isinstance(data['age'], int):
        raise ValueError('age must be an integer')
    if data['age'] < 0:
        raise ValueError('age must be positive')
    # ... more validation ...
```

With Pydantic, it's automatic:
```python
class User(BaseModel):
    name: str
    age: int
```

## Quick Intro to Type Hints

Type hints tell Python (and Pydantic) what type of data to expect:

- `str` - Text: `"hello"`
- `int` - Whole numbers: `42`
- `float` - Decimal numbers: `3.14`
- `bool` - True/False: `True`
- `Optional[str]` - Text or None: `"hello"` or `None`
- `List[str]` - List of text: `["a", "b", "c"]`
- `Dict[str, int]` - Dictionary: `{"age": 25}`

## Installation

Run this cell to install Pydantic (if not already installed):

In [ ]:
!uv pip install pydantic

## Your First Pydantic Model

Let's create a simple model and see Pydantic in action!

In [1]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

# Create a valid person
person = Person(name="Alice", age=30)
print(f"Name: {person.name}")
print(f"Age: {person.age}")
print(f"\nModel as dict: {person.model_dump()}")
print(f"Model as JSON: {person.model_dump_json()}")

Name: Alice
Age: 30

Model as dict: {'name': 'Alice', 'age': 30}
Model as JSON: {"name":"Alice","age":30}


---

# 2. Basic Models

## Creating Models with BaseModel

All Pydantic models inherit from `BaseModel`. Let's explore the key features:

In [2]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
    email: str
    age: int
    is_active: bool = True  # Default value

# Creating a user
user = User(
    name="John Doe",
    email="john@example.com",
    age=25
)

print(user)
print(f"\nName: {user.name}")
print(f"Email: {user.email}")
print(f"Active: {user.is_active}")  # Uses default value

name='John Doe' email='john@example.com' age=25 is_active=True

Name: John Doe
Email: john@example.com
Active: True


## Automatic Type Conversion

Pydantic automatically converts compatible types:

In [3]:
# String "25" gets converted to integer 25
user2 = User(name="Jane", email="jane@example.com", age="25")
print(f"Age: {user2.age} (type: {type(user2.age).__name__})")

# String "true" gets converted to boolean True
user3 = User(name="Bob", email="bob@example.com", age=30, is_active="yes")
print(f"Active: {user3.is_active} (type: {type(user3.is_active).__name__})")

Age: 25 (type: int)
Active: True (type: bool)


## Validation Errors

When data doesn't match the expected type, Pydantic raises clear validation errors:

In [4]:
from pydantic import ValidationError

try:
    # This will fail - age cannot be a string like "abc"
    invalid_user = User(name="Test", email="test@example.com", age="abc")
except ValidationError as e:
    print("❌ Validation failed!")
    print(e)

❌ Validation failed!
1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [5]:
try:
    # Missing required field 'email'
    incomplete_user = User(name="Test", age=25)
except ValidationError as e:
    print("❌ Validation failed!")
    print(e)

❌ Validation failed!
1 validation error for User
email
  Field required [type=missing, input_value={'name': 'Test', 'age': 25}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing


## Working with Models

### Converting to Dictionary and JSON

In [6]:
user = User(name="Alice", email="alice@example.com", age=28)

# Convert to dictionary
user_dict = user.model_dump()
print("As dictionary:")
print(user_dict)
print(f"Type: {type(user_dict)}")

# Convert to JSON string
user_json = user.model_dump_json()
print("\nAs JSON:")
print(user_json)
print(f"Type: {type(user_json)}")

As dictionary:
{'name': 'Alice', 'email': 'alice@example.com', 'age': 28, 'is_active': True}
Type: <class 'dict'>

As JSON:
{"name":"Alice","email":"alice@example.com","age":28,"is_active":true}
Type: <class 'str'>


### Creating Models from Dictionaries and JSON

In [7]:
# From dictionary
data = {"name": "Bob", "email": "bob@example.com", "age": 35}
user_from_dict = User(**data)  # ** unpacks the dictionary
print("From dict:", user_from_dict)

# From JSON string
json_string = '{"name": "Charlie", "email": "charlie@example.com", "age": 40}'
user_from_json = User.model_validate_json(json_string)
print("From JSON:", user_from_json)

From dict: name='Bob' email='bob@example.com' age=35 is_active=True
From JSON: name='Charlie' email='charlie@example.com' age=40 is_active=True


## Example: Product Model

Let's create a more practical example - an e-commerce product:

In [8]:
class Product(BaseModel):
    name: str
    price: float
    quantity: int
    in_stock: bool = True
    description: str = ""  # Optional with empty string default

# Create a product
laptop = Product(
    name="Gaming Laptop",
    price=999.99,
    quantity=50,
    description="High-performance laptop for gaming"
)

print(laptop)
print(f"\nTotal value: ${laptop.price * laptop.quantity:.2f}")

name='Gaming Laptop' price=999.99 quantity=50 in_stock=True description='High-performance laptop for gaming'

Total value: $49999.50


💡 **Try it yourself**: Create a product for something you'd like to buy!

In [ ]:
# Your code here


---

# 3. Field Types & Validation

Pydantic provides powerful field validation through the `Field` function and special types.

## String Field Constraints

In [9]:
from pydantic import BaseModel, Field

class UserRegistration(BaseModel):
    username: str = Field(min_length=3, max_length=20)
    password: str = Field(min_length=8)
    bio: str = Field(default="", max_length=500)

# Valid registration
user = UserRegistration(
    username="john_doe",
    password="securePass123",
    bio="I love coding!"
)
print("✅ Valid:", user)

✅ Valid: username='john_doe' password='securePass123' bio='I love coding!'


In [10]:
try:
    # Username too short
    invalid_user = UserRegistration(username="ab", password="password123")
except ValidationError as e:
    print("❌ Validation failed!")
    print(e)

❌ Validation failed!
1 validation error for UserRegistration
username
  String should have at least 3 characters [type=string_too_short, input_value='ab', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short


In [11]:
try:
    # Password too short
    invalid_user = UserRegistration(username="john", password="pass")
except ValidationError as e:
    print("❌ Validation failed!")
    print(e)

❌ Validation failed!
1 validation error for UserRegistration
password
  String should have at least 8 characters [type=string_too_short, input_value='pass', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short


## Numeric Field Constraints

In [12]:
class ProductListing(BaseModel):
    name: str
    price: float = Field(gt=0)  # Greater than 0
    quantity: int = Field(ge=0)  # Greater than or equal to 0
    discount_percent: float = Field(ge=0, le=100)  # Between 0 and 100
    rating: float = Field(ge=1.0, le=5.0)  # Rating from 1 to 5

# Valid product
product = ProductListing(
    name="Wireless Mouse",
    price=29.99,
    quantity=100,
    discount_percent=15.0,
    rating=4.5
)
print("✅ Valid:", product)

✅ Valid: name='Wireless Mouse' price=29.99 quantity=100 discount_percent=15.0 rating=4.5


In [13]:
try:
    # Negative price
    invalid_product = ProductListing(
        name="Item",
        price=-10.0,
        quantity=5,
        discount_percent=10,
        rating=4.0
    )
except ValidationError as e:
    print("❌ Validation failed!")
    print(e)

❌ Validation failed!
1 validation error for ProductListing
price
  Input should be greater than 0 [type=greater_than, input_value=-10.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


## Email and URL Validation

Pydantic provides special types for common validation patterns:

In [14]:
from pydantic import BaseModel, EmailStr, HttpUrl
from typing import Optional

class UserProfile(BaseModel):
    username: str
    email: EmailStr  # Validates email format
    website: Optional[HttpUrl] = None  # Validates URL format
    age: int = Field(ge=13, le=120)  # Age restrictions

# Valid profile
profile = UserProfile(
    username="alice",
    email="alice@example.com",
    website="https://alice.dev",
    age=25
)
print("✅ Valid:", profile)

✅ Valid: username='alice' email='alice@example.com' website=HttpUrl('https://alice.dev/') age=25


In [15]:
try:
    # Invalid email
    invalid_profile = UserProfile(
        username="bob",
        email="not-an-email",
        age=30
    )
except ValidationError as e:
    print("❌ Validation failed!")
    print(e)

❌ Validation failed!
1 validation error for UserProfile
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='not-an-email', input_type=str]


## Optional Fields

In [16]:
from typing import Optional

class CustomerInfo(BaseModel):
    name: str  # Required
    email: EmailStr  # Required
    phone: Optional[str] = None  # Optional - can be None
    address: str = ""  # Optional - default to empty string

# Without optional fields
customer1 = CustomerInfo(name="John", email="john@example.com")
print("Customer 1:", customer1)

# With optional fields
customer2 = CustomerInfo(
    name="Jane",
    email="jane@example.com",
    phone="+1234567890",
    address="123 Main St"
)
print("Customer 2:", customer2)

Customer 1: name='John' email='john@example.com' phone=None address=''
Customer 2: name='Jane' email='jane@example.com' phone='+1234567890' address='123 Main St'


## DateTime and UUID Fields

In [17]:
from datetime import datetime
from uuid import UUID, uuid4

class Event(BaseModel):
    event_id: UUID
    name: str
    created_at: datetime
    starts_at: datetime

# Create an event
event = Event(
    event_id=uuid4(),
    name="Python Workshop",
    created_at=datetime.now(),
    starts_at="2024-12-25T10:00:00"  # String automatically parsed
)

print(event)
print(f"\nEvent ID type: {type(event.event_id)}")
print(f"Start time type: {type(event.starts_at)}")

event_id=UUID('5f1e5653-c995-4e1e-ae26-8db1d671001b') name='Python Workshop' created_at=datetime.datetime(2025, 12, 15, 13, 20, 20, 414598) starts_at=datetime.datetime(2024, 12, 25, 10, 0)

Event ID type: <class 'uuid.UUID'>
Start time type: <class 'datetime.datetime'>


---

# 4. Nested Models & Complex Types

Real-world data is often hierarchical. Pydantic makes it easy to work with nested structures.

## Nested Models

In [18]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str
    country: str = "USA"

class Customer(BaseModel):
    name: str
    email: EmailStr
    address: Address  # Nested model

# Create a customer with address
customer = Customer(
    name="John Doe",
    email="john@example.com",
    address={
        "street": "123 Main St",
        "city": "San Francisco",
        "state": "CA",
        "zip_code": "94105"
    }
)

print(customer)
print(f"\nCity: {customer.address.city}")
print(f"\nAs dict:\n{customer.model_dump()}")

name='John Doe' email='john@example.com' address=Address(street='123 Main St', city='San Francisco', state='CA', zip_code='94105', country='USA')

City: San Francisco

As dict:
{'name': 'John Doe', 'email': 'john@example.com', 'address': {'street': '123 Main St', 'city': 'San Francisco', 'state': 'CA', 'zip_code': '94105', 'country': 'USA'}}


## Lists of Models

In [19]:
from typing import List

class OrderItem(BaseModel):
    product_name: str
    quantity: int = Field(gt=0)
    price: float = Field(gt=0)
    
    @property
    def total(self) -> float:
        return self.quantity * self.price

class Order(BaseModel):
    order_id: str
    customer_name: str
    items: List[OrderItem]  # List of OrderItem models
    
    @property
    def total_amount(self) -> float:
        return sum(item.total for item in self.items)

# Create an order with multiple items
order = Order(
    order_id="ORD-12345",
    customer_name="Alice",
    items=[
        {"product_name": "Laptop", "quantity": 1, "price": 999.99},
        {"product_name": "Mouse", "quantity": 2, "price": 29.99},
        {"product_name": "Keyboard", "quantity": 1, "price": 89.99}
    ]
)

print(order)
print(f"\nNumber of items: {len(order.items)}")
print(f"Total amount: ${order.total_amount:.2f}")

# Access individual items
for item in order.items:
    print(f"  {item.product_name}: {item.quantity} x ${item.price} = ${item.total}")

order_id='ORD-12345' customer_name='Alice' items=[OrderItem(product_name='Laptop', quantity=1, price=999.99), OrderItem(product_name='Mouse', quantity=2, price=29.99), OrderItem(product_name='Keyboard', quantity=1, price=89.99)]

Number of items: 3
Total amount: $1149.96
  Laptop: 1 x $999.99 = $999.99
  Mouse: 2 x $29.99 = $59.98
  Keyboard: 1 x $89.99 = $89.99


## Dictionaries with Models

In [20]:
from typing import Dict

class Student(BaseModel):
    name: str
    grade: float = Field(ge=0, le=100)

class Classroom(BaseModel):
    class_name: str
    students: Dict[str, Student]  # student_id -> Student

# Create a classroom
classroom = Classroom(
    class_name="Python 101",
    students={
        "S001": {"name": "Alice", "grade": 95.5},
        "S002": {"name": "Bob", "grade": 87.3},
        "S003": {"name": "Charlie", "grade": 92.1}
    }
)

print(classroom)
print(f"\nStudent S001: {classroom.students['S001'].name} - {classroom.students['S001'].grade}")

class_name='Python 101' students={'S001': Student(name='Alice', grade=95.5), 'S002': Student(name='Bob', grade=87.3), 'S003': Student(name='Charlie', grade=92.1)}

Student S001: Alice - 95.5


## Parsing JSON Data

A common use case is parsing JSON from APIs:

In [21]:
# Simulate JSON response from a weather API
json_response = '''
{
    "city": "San Francisco",
    "temperature": 18.5,
    "conditions": "Partly Cloudy",
    "forecast": [
        {"day": "Monday", "high": 20, "low": 15},
        {"day": "Tuesday", "high": 22, "low": 16},
        {"day": "Wednesday", "high": 19, "low": 14}
    ]
}
'''

class DayForecast(BaseModel):
    day: str
    high: float
    low: float

class WeatherData(BaseModel):
    city: str
    temperature: float
    conditions: str
    forecast: List[DayForecast]

# Parse JSON directly into model
weather = WeatherData.model_validate_json(json_response)

print(f"Weather in {weather.city}: {weather.temperature}°C - {weather.conditions}")
print("\nForecast:")
for day in weather.forecast:
    print(f"  {day.day}: High {day.high}°C, Low {day.low}°C")

Weather in San Francisco: 18.5°C - Partly Cloudy

Forecast:
  Monday: High 20.0°C, Low 15.0°C
  Tuesday: High 22.0°C, Low 16.0°C
  Wednesday: High 19.0°C, Low 14.0°C


## Deeply Nested Example

In [22]:
class Author(BaseModel):
    name: str
    email: EmailStr

class Comment(BaseModel):
    author: Author
    text: str
    created_at: datetime

class BlogPost(BaseModel):
    title: str
    content: str
    author: Author
    comments: List[Comment]
    tags: List[str]

# Create a blog post with nested data
blog_post = BlogPost(
    title="Getting Started with Pydantic",
    content="Pydantic is an amazing library...",
    author={"name": "John Doe", "email": "john@example.com"},
    comments=[
        {
            "author": {"name": "Alice", "email": "alice@example.com"},
            "text": "Great tutorial!",
            "created_at": "2024-01-15T10:30:00"
        },
        {
            "author": {"name": "Bob", "email": "bob@example.com"},
            "text": "Very helpful, thanks!",
            "created_at": "2024-01-15T11:45:00"
        }
    ],
    tags=["python", "pydantic", "tutorial"]
)

print(f"Post: {blog_post.title}")
print(f"By: {blog_post.author.name}")
print(f"Comments: {len(blog_post.comments)}")
print(f"Tags: {', '.join(blog_post.tags)}")

Post: Getting Started with Pydantic
By: John Doe
Comments: 2
Tags: python, pydantic, tutorial


---

# 5. Custom Validators

Sometimes you need custom validation logic beyond basic type checking.

## Field Validators

In [23]:
from pydantic import BaseModel, field_validator

class UserAccount(BaseModel):
    username: str
    password: str
    
    @field_validator('username')
    @classmethod
    def username_alphanumeric(cls, v: str) -> str:
        if not v.isalnum():
            raise ValueError('Username must be alphanumeric')
        return v
    
    @field_validator('password')
    @classmethod
    def password_strength(cls, v: str) -> str:
        if len(v) < 8:
            raise ValueError('Password must be at least 8 characters')
        if not any(char.isdigit() for char in v):
            raise ValueError('Password must contain at least one digit')
        if not any(char.isupper() for char in v):
            raise ValueError('Password must contain at least one uppercase letter')
        return v

# Valid account
account = UserAccount(username="john123", password="SecurePass1")
print("✅ Valid:", account)

✅ Valid: username='john123' password='SecurePass1'


In [24]:
try:
    # Invalid username (has special character)
    invalid_account = UserAccount(username="john@123", password="SecurePass1")
except ValidationError as e:
    print("❌ Validation failed!")
    print(e)

❌ Validation failed!
1 validation error for UserAccount
username
  Value error, Username must be alphanumeric [type=value_error, input_value='john@123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [25]:
try:
    # Weak password (no uppercase)
    invalid_account = UserAccount(username="john123", password="password1")
except ValidationError as e:
    print("❌ Validation failed!")
    print(e)

❌ Validation failed!
1 validation error for UserAccount
password
  Value error, Password must contain at least one uppercase letter [type=value_error, input_value='password1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


## Model Validators

Model validators allow you to validate multiple fields together:

In [26]:
from pydantic import BaseModel, model_validator
from typing import Any

class DateRange(BaseModel):
    start_date: datetime
    end_date: datetime
    
    @model_validator(mode='after')
    def check_dates(self) -> 'DateRange':
        if self.start_date >= self.end_date:
            raise ValueError('end_date must be after start_date')
        return self

# Valid date range
date_range = DateRange(
    start_date="2024-01-01T00:00:00",
    end_date="2024-12-31T23:59:59"
)
print("✅ Valid:", date_range)

✅ Valid: start_date=datetime.datetime(2024, 1, 1, 0, 0) end_date=datetime.datetime(2024, 12, 31, 23, 59, 59)


In [27]:
try:
    # Invalid - end before start
    invalid_range = DateRange(
        start_date="2024-12-31T00:00:00",
        end_date="2024-01-01T00:00:00"
    )
except ValidationError as e:
    print("❌ Validation failed!")
    print(e)

❌ Validation failed!
1 validation error for DateRange
  Value error, end_date must be after start_date [type=value_error, input_value={'start_date': '2024-12-3...: '2024-01-01T00:00:00'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


## Password Confirmation Example

In [28]:
class PasswordReset(BaseModel):
    password: str = Field(min_length=8)
    password_confirmation: str
    
    @model_validator(mode='after')
    def passwords_match(self) -> 'PasswordReset':
        if self.password != self.password_confirmation:
            raise ValueError('Passwords do not match')
        return self

# Valid - passwords match
reset = PasswordReset(
    password="NewSecurePass123",
    password_confirmation="NewSecurePass123"
)
print("✅ Valid: Passwords match")

✅ Valid: Passwords match


In [29]:
try:
    # Invalid - passwords don't match
    invalid_reset = PasswordReset(
        password="Password123",
        password_confirmation="DifferentPassword123"
    )
except ValidationError as e:
    print("❌ Validation failed!")
    print(e)

❌ Validation failed!
1 validation error for PasswordReset
  Value error, Passwords do not match [type=value_error, input_value={'password': 'Password123... 'DifferentPassword123'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


---

# 6. Configuration Management

Pydantic is excellent for managing application configuration and environment variables.

## Basic Configuration Pattern

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

class DatabaseConfig(BaseModel):
    host: str = "localhost"
    port: int = Field(default=5432, ge=1, le=65535)
    username: str
    password: str
    database_name: str
    
    @property
    def connection_string(self) -> str:
        return f"postgresql://{self.username}:{self.password}@{self.host}:{self.port}/{self.database_name}"

# Create configuration
db_config = DatabaseConfig(
    username="myuser",
    password="mypassword",
    database_name="mydatabase"
)

print(f"Database: {db_config.database_name}")
print(f"Host: {db_config.host}:{db_config.port}")
print(f"Connection: {db_config.connection_string}")

## Application Settings

In [ ]:
class AppSettings(BaseModel):
    # Application info
    app_name: str = "My Application"
    version: str = "1.0.0"
    debug: bool = False
    
    # Database
    database: DatabaseConfig
    
    # API settings
    api_key: Optional[str] = None
    api_timeout: int = Field(default=30, ge=1)
    max_connections: int = Field(default=100, ge=1)

# Create application settings
settings = AppSettings(
    app_name="Todo API",
    debug=True,
    database={
        "username": "admin",
        "password": "secret",
        "database_name": "todos"
    },
    api_key="sk-abc123"
)

print(settings.model_dump_json(indent=2))

## Loading from Environment Variables

For Pydantic v2, we use `pydantic-settings` package for `BaseSettings`:

In [ ]:
# Install pydantic-settings if needed
!pip install pydantic-settings -q

In [ ]:
import os
from pydantic_settings import BaseSettings

# Set some environment variables for demonstration
os.environ['DATABASE_HOST'] = 'prod-db.example.com'
os.environ['DATABASE_PORT'] = '5432'
os.environ['DATABASE_USER'] = 'prod_user'
os.environ['DATABASE_PASSWORD'] = 'prod_password'
os.environ['DATABASE_NAME'] = 'production'
os.environ['API_KEY'] = 'sk-production-key'
os.environ['DEBUG_MODE'] = 'False'

class Settings(BaseSettings):
    # These will be loaded from environment variables
    database_host: str = "localhost"
    database_port: int = 5432
    database_user: str
    database_password: str
    database_name: str
    api_key: str
    debug_mode: bool = True

# Automatically loads from environment variables
settings = Settings()

print(f"Database Host: {settings.database_host}")
print(f"Database Port: {settings.database_port}")
print(f"Database User: {settings.database_user}")
print(f"Debug Mode: {settings.debug_mode}")
print(f"API Key: {settings.api_key[:10]}...")

## Loading from .env Files

In [ ]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class AppConfig(BaseSettings):
    database_host: str
    database_port: int
    database_user: str
    database_password: str
    database_name: str
    api_key: str
    debug_mode: bool
    
    model_config = SettingsConfigDict(
        env_file=".env",
        env_file_encoding="utf-8",
        case_sensitive=False
    )

# This would load from .env file if it exists
# For now, we'll use the environment variables we set
config = AppConfig()
print(f"Loaded configuration from environment:")
print(f"  Database: {config.database_user}@{config.database_host}:{config.database_port}/{config.database_name}")
print(f"  Debug: {config.debug_mode}")

💡 **Note**: In a real application, you would:
1. Create a `.env` file (use `.env.example` as a template)
2. Add `.env` to your `.gitignore`
3. Load settings once at application startup
4. Never commit secrets to version control!